In [1]:
from alpaca.data.historical import CryptoHistoricalDataClient
from alpaca.data.requests import CryptoBarsRequest
from alpaca.data.timeframe import TimeFrame
from kafka import KafkaProducer
from json import dumps
import pandas as pd
import schedule
import time
from datetime import datetime

In [4]:
from alpaca.data.historical import CryptoHistoricalDataClient
from alpaca.data.requests import CryptoBarsRequest
from alpaca.data.timeframe import TimeFrame
from kafka import KafkaProducer
from json import dumps
import pandas as pd
import schedule
import time
from datetime import datetime, timedelta

# Initialize Kafka Producer
producer = KafkaProducer(
    bootstrap_servers=['18.219.236.123:9092'],
    value_serializer=lambda x: dumps(x).encode('utf-8')
)

# Initialize Alpaca Client
client = CryptoHistoricalDataClient()

def fetch_and_send_crypto_data():
    """
    Fetch crypto data and send each symbol separately
    """
    try:
        # Calculate time range (simulating real-time by moving window)
        end_time = datetime.utcnow()
        start_time = end_time - timedelta(minutes=5)  # Get last 5 minutes of data
        
        request_params = CryptoBarsRequest(
            symbol_or_symbols=["DOGE/USD", "ETH/USD", "BTC/USD"],
            timeframe=TimeFrame.Minute,
            start=start_time.isoformat() + "Z",
            end=end_time.isoformat() + "Z"
        )
        
        # Get the data
        bars = client.get_crypto_bars(request_params)
        df = bars.df.reset_index()
        
        # Process each cryptocurrency separately
        for symbol in ["DOGE/USD", "ETH/USD", "BTC/USD"]:
            symbol_data = df[df['symbol'] == symbol].iloc[-1]  # Get the latest data point
            
            # Create message for this symbol
            message = {
                'symbol': symbol,
                'timestamp': symbol_data['timestamp'].isoformat() + 'Z',
                'close': float(symbol_data['close'])
            }
            
            # Send to Kafka
            producer.send('crypto_realtime', value=message)
            producer.flush()
            print(f"Sent {symbol} data: {message}")
            
    except Exception as e:
        print(f"Error in fetch_and_send_crypto_data: {e}")

def start_producer():
    print("Starting crypto data producer...")
    
    # Run immediately at start
    fetch_and_send_crypto_data()
    
    # Schedule to run every 30 seconds
    schedule.every(10).seconds.do(fetch_and_send_crypto_data)
    
    while True:
        try:
            schedule.run_pending()
            time.sleep(1)
        except KeyboardInterrupt:
            print("\nShutting down producer...")
            break
        except Exception as e:
            print(f"Error in scheduler: {e}")
            time.sleep(5)

if __name__ == "__main__":
    start_producer()

Starting crypto data producer...
Sent DOGE/USD data: {'symbol': 'DOGE/USD', 'timestamp': '2024-11-23T03:31:00+00:00Z', 'close': 0.431985}
Sent ETH/USD data: {'symbol': 'ETH/USD', 'timestamp': '2024-11-23T03:30:00+00:00Z', 'close': 3344.934}
Sent BTC/USD data: {'symbol': 'BTC/USD', 'timestamp': '2024-11-23T03:28:00+00:00Z', 'close': 98926.8}
Sent DOGE/USD data: {'symbol': 'DOGE/USD', 'timestamp': '2024-11-23T03:31:00+00:00Z', 'close': 0.431985}
Sent ETH/USD data: {'symbol': 'ETH/USD', 'timestamp': '2024-11-23T03:30:00+00:00Z', 'close': 3344.934}
Sent BTC/USD data: {'symbol': 'BTC/USD', 'timestamp': '2024-11-23T03:28:00+00:00Z', 'close': 98926.8}
Sent DOGE/USD data: {'symbol': 'DOGE/USD', 'timestamp': '2024-11-23T03:31:00+00:00Z', 'close': 0.431835}
Sent ETH/USD data: {'symbol': 'ETH/USD', 'timestamp': '2024-11-23T03:30:00+00:00Z', 'close': 3344.934}
Sent BTC/USD data: {'symbol': 'BTC/USD', 'timestamp': '2024-11-23T03:28:00+00:00Z', 'close': 98926.8}
Sent DOGE/USD data: {'symbol': 'DOGE/